# (실습-9) LLM 응용 구현 - JSON mode, function calling 실습

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 LLM의 JSON mode와 function calling 기능에 대하여 알아본다. <br>
  각 기능에 대하여 어떻게 응용에서 활용할 수 있는지 이해한다. <br>
2) 수강 목표
  * LLM 응용 구현을 위한 LLM 프롬프트 엔지니어링 기법을 이해한다.
  * OpenAI API로 제공하는 LLM의 JSON mode를 사용할 수 있다.
  * OpenAI API로 제공하는 LLM의 function calling 기능을 사용할 수 있다.

### 실습 목차
* 1. JSON mode 활용
* 2. Function calling 활용

### 환경 설정
OpenAI 패키지를 설치한다. <br>
실험의 편의를 위해 테스트를 위한 공통함수를 정의한다.

In [50]:
# OpenAI Python 패키지 설치
# !pip install openai

In [1]:
# OpenAI 라이브러리 및 필요한 모듈을 가져옵니다.
import os
import json
from openai import OpenAI

# OpenAI API 키를 환경변수에서 설정합니다.
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()
# 사용할 모델을 설정합니다. 여기서는 gpt-3.5-turbo-1106 모델을 사용합니다.
#llm_model = "gpt-4-1106-preview"
llm_model = "gpt-3.5-turbo-1106"


In [2]:
# 각 테스트 케이스에 대해 openai API를 수행하는 함수를 정의합니다.
def test(case, persona, response_format=None):
    messages = [
        {"role": "system", "content": persona},
        {"role": "user", "content": case}
    ]

    print(f'Test case: {case}')

    # response_format이 지정되었으면 해당 형식으로 결과를 반환합니다.
    if response_format:
        result = client.chat.completions.create(
            model=llm_model,
            messages=messages,
            temperature=0.1,
            response_format=response_format,
            seed=1
        )
    else:
        result = client.chat.completions.create(
            model=llm_model,
            messages=messages,
            temperature=0.1,
            seed=1
        )

    # 결과를 출력합니다.
    print(f'Result: \n{result.choices[0].message.content}')

## 1. JSON mode 활용




이 코드는 OpenAI의 API를 사용하여 주어진 리뷰의 감성을 '긍정', '부정', '중립'으로 분류하는 실습입니다. 사용자가 제공한 리뷰 텍스트를 바탕으로 감성을 분석하고, 결과는 일반 텍스트 혹은 JSON 형식으로 출력합니다. 코드는 다양한 리뷰 샘플에 대해 이러한 감성 분석을 수행하는 과정을 보여줍니다.



**JSON출력을 명시하는 경우와 그렇지 않은 경우를 비교해봅시다**

In [3]:

# 테스트 할 리뷰들을 정의합니다.
test_case = [
"가성비 굳!",
"다시는 이제품을 사나봐라",
"재구입 의사 있습니다.",
"비추는 아니지만 그렇다고 추천할만 하지는 않음"
]

# 감성 분석을 수행할 때 사용할 지침(페르소나, 프롬프트)을 정의합니다.
persona = """
## Role: 리뷰 감성 분류

## Instruction
- 주어진 리뷰가 '긍정', '부정', '중립'인지 판단해줘.
- 추가적인 설명은 필요없고 감성 분류 결과만 알려줘.
"""

# 결과를 JSON 형식으로 출력하도록 지침을 수정합니다.
persona_json = """
## Role: 리뷰 감성 분류

## Instruction
- 주어진 리뷰가 '긍정', '부정', '중립'인지 판단해줘.
- 추가적인 설명은 필요없고 감성 분류 결과만 알려줘.
- 결과는 JSON 형태로 출력해줘.

## Output Format
{"Sentiment": $sentiment}
"""


In [4]:
# JSON 출력 명시하지 않은 경우
for tc in test_case:
    test(tc, persona)
    print('')

Test case: 가성비 굳!
Result: 
긍정적인 감성을 가진 리뷰로 분류됩니다.

Test case: 다시는 이제품을 사나봐라
Result: 
부정

Test case: 재구입 의사 있습니다.
Result: 
긍정적인 감성입니다.

Test case: 비추는 아니지만 그렇다고 추천할만 하지는 않음
Result: 
중립



In [5]:
# JSON 출력 명시한 경우
for tc in test_case:
    test(tc, persona_json)
    print('')

Test case: 가성비 굳!
Result: 
{"Sentiment": "긍정"}

Test case: 다시는 이제품을 사나봐라
Result: 
{"Sentiment": "부정"}

Test case: 재구입 의사 있습니다.
Result: 
{"Sentiment": "긍정"}

Test case: 비추는 아니지만 그렇다고 추천할만 하지는 않음
Result: 
{"Sentiment": "중립"}



In [6]:
# JSON 출력 명시한 경우 + OpenAI JSON mode를 사용
for tc in test_case:
    test(tc, persona_json, {"type": "json_object"})
    print('')

Test case: 가성비 굳!
Result: 
{"Sentiment": "긍정"}

Test case: 다시는 이제품을 사나봐라
Result: 
{"Sentiment": "부정"}

Test case: 재구입 의사 있습니다.
Result: 
{"Sentiment": "긍정"}

Test case: 비추는 아니지만 그렇다고 추천할만 하지는 않음
Result: 
{"Sentiment": "중립"}



아래 코드는 OpenAI의 GPT 모델을 사용하여 주어진 텍스트의 의도를 분류하는 작업을 수행합니다. 텍스트의 의도는 '요약', '번역', '질답', '기타' 중 하나로 결정됩니다. 사용자의 입력에 따라 이러한 의도를 분석하고, 결과는 일반 텍스트 또는 JSON 형식으로 출력합니다. 코드는 다양한 텍스트 샘플에 대해 이 의도 분석 과정을 시연합니다.

**JSON출력을 명시하는 경우와 그렇지 않은 경우를 비교해봅시다**

In [7]:
test_case = [
"다음 뉴스 요약해줘.\n60만 패캐머가 선택한 강의, 이제 우리 회사에서 만나보세요! Fastcampus 강의로 취업부터 직무역량 강화까지! 전화, 카톡 상담가능. 강의자료 제공. 약 200개 인강 무제한 수강. 분야별 전문 강사진 강의. PC/태블릿/모바일 수강가능. 누적 수강생 20만명. 맞춤형 커리큘럼 제공.",
"아래 문장 번역 좀 해줄래? \nI love Fast Campus.",
"패스트캠퍼스가 뭐야?",
"안녕, 반가워!"
]

persona = """
## Role: 의도 분류

## Instruction
- 의도는 '요약', '번역', 질답', '기타' 네가지 중 하나로 결정해야 돼!
- 추가적인 설명은 필요없고 의도만 알려줘.
"""


persona_json = """
## Role: 의도 분류

## Instruction
- 의도는 '요약', '번역', 질답', '기타' 네가지 중 하나로 결정해야 돼!
- 추가적인 설명은 필요없고 의도만 알려줘.
- 결과는 JSON 형태로 출력해줘.

## Output Format
{"Intent": $intent}
"""


In [8]:
# JSON 출력 명시하지 않은 경우
for tc in test_case:
    test(tc, persona)
    print()

Test case: 다음 뉴스 요약해줘.
60만 패캐머가 선택한 강의, 이제 우리 회사에서 만나보세요! Fastcampus 강의로 취업부터 직무역량 강화까지! 전화, 카톡 상담가능. 강의자료 제공. 약 200개 인강 무제한 수강. 분야별 전문 강사진 강의. PC/태블릿/모바일 수강가능. 누적 수강생 20만명. 맞춤형 커리큘럼 제공.
Result: 
의도: 요약

Test case: 아래 문장 번역 좀 해줄래? 
I love Fast Campus.
Result: 
의도: 번역

Test case: 패스트캠퍼스가 뭐야?
Result: 
의도: 질답

Test case: 안녕, 반가워!
Result: 
기타



In [9]:
# JSON 출력 명시한 경우
for tc in test_case:
    test(tc, persona_json)
    print()

Test case: 다음 뉴스 요약해줘.
60만 패캐머가 선택한 강의, 이제 우리 회사에서 만나보세요! Fastcampus 강의로 취업부터 직무역량 강화까지! 전화, 카톡 상담가능. 강의자료 제공. 약 200개 인강 무제한 수강. 분야별 전문 강사진 강의. PC/태블릿/모바일 수강가능. 누적 수강생 20만명. 맞춤형 커리큘럼 제공.
Result: 
{"Intent": "요약"}

Test case: 아래 문장 번역 좀 해줄래? 
I love Fast Campus.
Result: 
{"Intent": "번역"}

Test case: 패스트캠퍼스가 뭐야?
Result: 
{"Intent": "질답"}

Test case: 안녕, 반가워!
Result: 
{"Intent": "기타"}



In [10]:
# JSON 출력 명시한 경우 + OpenAI JSON mode를 사용
for tc in test_case:
    test(tc, persona_json, {"type": "json_object"})
    print()

Test case: 다음 뉴스 요약해줘.
60만 패캐머가 선택한 강의, 이제 우리 회사에서 만나보세요! Fastcampus 강의로 취업부터 직무역량 강화까지! 전화, 카톡 상담가능. 강의자료 제공. 약 200개 인강 무제한 수강. 분야별 전문 강사진 강의. PC/태블릿/모바일 수강가능. 누적 수강생 20만명. 맞춤형 커리큘럼 제공.
Result: 
{"Intent": "요약"}

Test case: 아래 문장 번역 좀 해줄래? 
I love Fast Campus.
Result: 
{"Intent": "번역"}

Test case: 패스트캠퍼스가 뭐야?
Result: 
{"Intent": "질답"}

Test case: 안녕, 반가워!
Result: 
{"Intent": "기타"}



아래 코드는 OpenAI의 GPT 모델을 사용하여 사용자의 메시지를 분석하고 특정 정보를 추출하는 것을 목표로 합니다. 특히, 사용자의 메시지가 식당에 대한 검색 요청인지를 판단하고, 그 안에서 '식당 위치'와 '음식 종류'를 식별해내는 기능을 수행합니다. 결과는 일반 텍스트 형식 또는 JSON 형식으로 출력됩니다. 코드는 다양한 요청 예시에 대해 이러한 정보 추출 과정을 시연합니다.

**JSON출력을 명시하는 경우와 그렇지 않은 경우를 비교해봅시다**

In [64]:
test_case = [
"강남역 근처 맛집 추천해줘",
"청담동 일식집 찾아줘",
"제주에서 흑돼지 잘하는집 알려줘",
"제주로 여행가고 싶다"
]

persona = """
## Role: 질의 분석

## Instruction
- 주어진 질문으로부터 '식당 위치'와 '음식 종류'를 추출해줘.
- 추가적인 설명은 추출된 결과만 알려줘.
"""


persona_json = """
## Role: 질의 분석

## Instruction
- 사용자 메시지가 식당에 대한 검색 요청인지 아닌지 판단해줘.
- 주어진 질문으로부터 '식당 위치'와 '음식 종류'를 추출해줘.
- 추가적인 설명은 추출된 결과만 알려줘.
- 결과는 JSON 형태로 출력해줘.

## Output Format
{"검색 요청": true/false, "식당 위치": $location, "음식 종류": $food_kind}
"""

In [65]:
# JSON 출력 명시하지 않은 경우
for tc in test_case:
    test(tc, persona)
    print()

Test case: 강남역 근처 맛집 추천해줘
Result: 
식당 위치: 강남역 근처
음식 종류: 다양한 종류의 음식

강남역 근처에는 다양한 종류의 맛집이 있습니다. 강남역 주변에는 한식, 양식, 중식, 일식 등 다양한 음식 종류의 맛집이 있으니 원하는 음식 종류에 맞게 맛집을 찾아보시면 좋을 것 같습니다.

Test case: 청담동 일식집 찾아줘
Result: 
식당 위치: 청담동
음식 종류: 일식집

청담동에 위치한 일식집을 찾아드릴게요.

Test case: 제주에서 흑돼지 잘하는집 알려줘
Result: 
식당 위치: 제주도
음식 종류: 흑돼지 요리

추가적인 설명:
제주도는 흑돼지 요리로 유명한 곳으로, 맛있는 흑돼지 요리를 즐길 수 있는 다양한 식당이 있습니다. 특히 제주시와 서귀포시에는 많은 흑돼지 전문 음식점들이 있으니 방문해보시기를 추천드립니다.

Test case: 제주로 여행가고 싶다
Result: 
제주도는 정말 멋진 여행지에요! 제주도에는 다양한 음식이 있지만 특히 유명한 음식으로는 흑돼지고기, 해물죽, 갈치조림 등이 있어요. 또한, 제주시, 서귀포시 등 다양한 지역에 맛있는 음식점들이 있으니 현지 음식을 즐기며 여행을 즐기시면 좋을 것 같아요.



In [66]:
# JSON 출력 명시한 경우
for tc in test_case:
    test(tc, persona_json)
    print()

Test case: 강남역 근처 맛집 추천해줘
Result: 
{"검색 요청": true, "식당 위치": "강남역 근처", "음식 종류": null}

Test case: 청담동 일식집 찾아줘
Result: 
{"검색 요청": true, "식당 위치": "청담동", "음식 종류": "일식"}

Test case: 제주에서 흑돼지 잘하는집 알려줘
Result: 
{"검색 요청": true, "식당 위치": "제주", "음식 종류": "흑돼지"}

Test case: 제주로 여행가고 싶다
Result: 
{"검색 요청": false}



In [67]:
# JSON 출력 명시한 경우 + OpenAI JSON mode를 사용
for tc in test_case:
    test(tc, persona_json, {"type": "json_object"})
    print()

Test case: 강남역 근처 맛집 추천해줘
Result: 
{"검색 요청": true, "식당 위치": "강남역 근처", "음식 종류": null}

Test case: 청담동 일식집 찾아줘
Result: 
{"검색 요청": true, "식당 위치": "청담동", "음식 종류": "일식"}

Test case: 제주에서 흑돼지 잘하는집 알려줘
Result: 
{"검색 요청": true, "식당 위치": "제주", "음식 종류": "흑돼지"}

Test case: 제주로 여행가고 싶다
Result: 
{"검색 요청": false}



## 2. Function calling 활용

원하는 동작이 있을 때 JSON mode를 활용하여 구현할 수도 있지만 function calling을 활용하는 경우 다양한 조건에서 적합한 로직을 구현하는데 훨씬 수월합니다. <br>
아래는 function calling 사용하는 예시입니다. <br>
**JSON 출력을 명시하는 방식과 비교해 봅시다.**

In [11]:
persona_json1 = """
## Role: 질의 분석

## Instruction
- 주고받은 사용자 메시지를 분석하여 검색에 사용하기 적합한 최종 질의, 식당 위치, 음식 종류를 포함하여 아래와 같이 JSON 형태로 출력한다.

## Output Format
{"standalone_query": $standalone_query, "location": $location, "cuisine": $food_type}
"""

persona_json2 = """
## Role: 식당 검색 도우미

## Instruction
- 사용자의 요청이 식당 추천이나 검색인 경우에는 질의 분석 결과를 생성한다.
- 질의 분석 결과에 대해서는 지금까지 주고받은 전체 사용자 메시지를 분석하여 검색에 사용하기 적합한 최종 질의, 식당 위치, 음식 종류를 포함하여 아래와 같이 JSON 형태로 출력한다.
- 식당 추천이나 검색이 아닌 나머지 질의는 적절한 대답을 생성한다.

## Output Format
{"standalone_query": $standalone_query, "location": $location, "cuisine": $food_type}
"""

persona_function_calling = """
## Name: Good Place

## Role: 식당 검색 도우미

## Instruction
- 사용자의 요청이 식당 추천이나 검색인 경우에는 질의 분석 결과를 생성한다.
- 질의 분석 결과에 대해서는 지금까지 주고받은 전체 사용자 메시지를 분석하여 검색에 사용하기 적합한 최종 질의, 식당 위치, 음식 종류를 function calling 결과에 포함하여 출력한다.
- 식당 추천이나 검색이 아닌 나머지 질의는 적절한 대답을 생성한다.
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_restaurant",
            "description": "식당을 검색한다.",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "기존 메시지 히스토리를 감안한 검색에 사용하기 적합한 최종 질의"
                    },
                    "location": {
                        "type": "string",
                        "description": "서울, 제주, 강남역, 신사역 등 식당의 위치"
                    },
                    "cuisine": {
                        "type": "string",
                        "description": "음식의 종류",
                        "enum": ["한식", "일식", "중식", "태국음식", "이탈리안"]
                    }
                },
                "required": ["standalone_query", "location", "cuisine"],
                "type": "object"
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "detail_restaurant",
            "description": "식당 상세 정보 제공",
            "parameters": {
                "properties": {
                    "place_name": {
                        "type": "string",
                        "description": "식당 이름",
                    },
                    "question": {
                        "type": "string",
                        "description": "상세 정보 요청을 위한 질문"
                    }
                },
                "required": ["place_name", "question"],
                "type": "object"
            }
        }
    }
]



In [12]:
# 공통 함수 정의
def test(messages, persona, response_format=None):
    msg = [{"role": "system", "content": persona}] + messages

    if response_format:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            temperature=0,
            response_format=response_format,
            seed=1
        )
    else:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            temperature=0,
            seed=1
        )

    print(result.choices[0].message)

def test_with_function_calling(messages, persona, tools, tool_choice=None):
    msg = [{"role": "system", "content": persona}] + messages

    if tool_choice:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            tool_choice=tool_choice,
            temperature=0,
            seed=1
        )

    else:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            temperature=0,
            seed=1
        )
    print(result.choices[0].message)

In [13]:
messages = [
{"role": "user", "content": "너는 누구니"},
{"role": "assistant", "content": "저는 식당 검색 도우미 Good Place 라고 합니다."},
{"role": "user", "content": "강남역 근처 맛집 추천해줘"},
{"role": "assistant", "content": "강남역 근처 맛집 A, B, C 추천합니다."},
{"role": "user", "content": "일식집으로 알아봐줘"},
{"role": "assistant", "content": "일식집으로는 D, E, F가 있습니다."},
{"role": "user", "content": "아니다 갑자기 태국음식이 땡기네"} # implicit instruction
]

In [14]:
# Function calling 사용하지 않고 명시적으로 요청하는 예시 1
test(messages, persona_json1, {"type": "json_object"})

ChatCompletionMessage(content='\n{"standalone_query": "태국음식", "location": "강남역", "cuisine": "태국음식"}', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [15]:
# Function calling 사용하지 않고 명시적으로 요청하는 예시 2
test(messages, persona_json2, {"type": "json_object"})

ChatCompletionMessage(content='{"standalone_query": "태국음식", "location": "강남역", "cuisine": "태국음식"}', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [16]:
# Function calling 사용하지 않고 명시적으로 요청하는 예시 1 (첫번째 라인만 입력한 경우)
test(messages[:1], persona_json1, {"type": "json_object"})

ChatCompletionMessage(content='{"standalone_query": "너는 누구니", "location": null, "cuisine": null}', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [17]:
# Function calling 사용하지 않고 명시적으로 요청하는 예시 2 (첫번째 라인만 입력한 경우)
test(messages[:1], persona_json2, {"type": "json_object"})

ChatCompletionMessage(content='{"standalone_query": "나는 식당 검색 도우미입니다.", "location": null, "cuisine": null}', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [18]:
# Function calling 사용하고 첫번째 라인만 입력한 경우
test_with_function_calling(messages[:1], persona_function_calling, tools)

ChatCompletionMessage(content='저는 "Good Place"입니다. 식당을 검색하거나 추천해드리는데 도움을 드릴 수 있어요. 원하시는 식당이나 음식에 대해 물어보시면 도와드릴게요!', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [19]:
# Function calling 사용하고 전체 메시지 입력한 경우
test_with_function_calling(messages, persona_function_calling, tools)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1PWEFS43r9WjtXj4x6ckdcO0', function=Function(arguments='{"standalone_query":"태국음식","location":"강남역","cuisine":"태국음식"}', name='search_restaurant'), type='function')])


In [20]:
# 필요한 경우 특정 function을 항상 사용하도록 명시할 수 있음
test_with_function_calling(messages, persona_function_calling, tools, {"type": "function", "function": {"name": "search_restaurant"}})

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_F5zTrd7uCBk21VZ4yhjkxBQQ', function=Function(arguments='{"standalone_query":"태국음식","location":"강남역","cuisine":"태국음식"}', name='search_restaurant'), type='function')])


In [21]:
messages = [
{"role": "user", "content": "너는 누구니"},
{"role": "assistant", "content": "저는 식당 검색 도우미 Good Place 라고 합니다."},
{"role": "user", "content": "파파야리프 저녁에 예약하면 주차 가능해?"}
]

In [22]:
# Function calling을 통해 원하는 function이 선택되는지 확인
test_with_function_calling(messages, persona_function_calling, tools)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DCwq6iN2tdoHTMeLiAO0I0G6', function=Function(arguments='{"place_name":"파파야리프","question":"주차 가능 여부와 예약 정보를 알려주세요."}', name='detail_restaurant'), type='function')])


#Reference

## Required Package

openai==1.7.2
